# Блок 1( на 1 и 2 этапах из всех во все)

In [ ]:
import pandas as pd
import numpy as np
import time

# Замеряем время выполнения
start_time = time.time()

# 1. Загружаем данные из Excel
df = pd.read_excel('Для Анализа Излишко Недостач.xlsx')

# 2. Проверяем количество столбцов
print("Количество столбцов:", len(df.columns))
print("Названия столбцов:", df.columns.tolist())

# 3. Переименовываем столбцы (10 столбцов)
df.columns = [
    'Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 
    'Сумма_продаж_тыс_руб', 'Остатки_тыс_руб', 'Ср_дневн_продажи_тыс_руб', 
    'Запас_в_днях', 'Кол_во_продано_шт', 'Торговых_дней'
]

# 4. Проверяем данные на пропуски и некорректные значения
print("\nПроверка данных на пропуски:")
print(df[['Остатки_шт', 'Запас_в_днях', 'Кол_во_продано_шт', 'Торговых_дней']].isnull().sum())
print("\nПервые 5 строк данных:")
print(df[['Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 'Запас_в_днях', 'Кол_во_продано_шт', 'Торговых_дней']].head())

# 5. Убедимся, что "Запас_в_днях" числовой
df['Запас_в_днях'] = pd.to_numeric(df['Запас_в_днях'], errors='coerce')

# 6. Рассчитываем стоимость одной единицы товара (в тыс. руб.)
df['Стоимость_за_единицу_тыс_руб'] = df['Остатки_тыс_руб'] / df['Остатки_шт']
df['Стоимость_за_единицу_тыс_руб'] = df['Стоимость_за_единицу_тыс_руб'].replace([np.inf, -np.inf, np.nan], 0)

# Для товаров с нулевыми остатками используем среднюю стоимость по товару
mean_price_per_item = df[df['Стоимость_за_единицу_тыс_руб'] > 0].groupby('Товар')['Стоимость_за_единицу_тыс_руб'].mean()
df = df.merge(mean_price_per_item.rename('Средняя_стоимость_за_единицу_тыс_руб'), on='Товар', how='left')
df['Стоимость_за_единицу_тыс_руб'] = np.where(
    df['Остатки_шт'] == 0,
    df['Средняя_стоимость_за_единицу_тыс_руб'].fillna(0),
    df['Стоимость_за_единицу_тыс_руб']
)

# 7. Определяем магазины с излишками и недостатками
df['Излишки'] = df['Запас_в_днях'] > 90
df['Недостаток'] = df['Запас_в_днях'] < 90

# 8. Рассчитываем средние дневные продажи в штуках
df['Ср_дневн_продажи_шт'] = df['Кол_во_продано_шт'] / df['Торговых_дней']
df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace([np.inf, -np.inf, np.nan], 0)

# Если Ср_дневн_продажи_шт = 0, используем минимальное ненулевое значение
min_sales_rate = df[df['Ср_дневн_продажи_шт'] > 0]['Ср_дневн_продажи_шт'].min()
if pd.notna(min_sales_rate):
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, min_sales_rate)
else:
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, 0.01)

# 9. Рассчитываем желаемые остатки, лишний и недостающий товар
df['Желаемые_остатки_шт'] = (df['Ср_дневн_продажи_шт'] * 90).round()
df['Лишний_товар_шт'] = np.where(df['Излишки'], (df['Остатки_шт'] - df['Желаемые_остатки_шт']).clip(lower=0), 0)
df['Недостающий_товар_шт'] = np.where(df['Недостаток'], (df['Желаемые_остатки_шт'] - df['Остатки_шт']).clip(lower=0), 0)

# 10. Выводим магазины с излишками
print("\nМагазины с излишками (запас > 90 дней):")
excess_stores = df[df['Излишки']][['Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 'Ср_дневн_продажи_шт', 'Запас_в_днях', 'Лишний_товар_шт', 'Стоимость_за_единицу_тыс_руб']]
print(excess_stores)

# 11. Выводим магазины с недостатком
print("\nМагазины с недостатком (запас < 90 дней):")
shortage_stores = df[df['Недостаток']][['Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 'Ср_дневн_продажи_шт', 'Запас_в_днях', 'Недостающий_товар_шт', 'Стоимость_за_единицу_тыс_руб']]
print(shortage_stores)

# 12. Перераспределяем товар с учётом условия "минимум 3 штуки" и отслеживаем неперемещённый товар
recommendations = []
remaining_excess = []
MIN_TRANSFER_QTY = 3                                                                     # Минимальное количество для перемещения

# Группируем по товару
grouped = df.groupby('Товар')

for item, group in grouped:
    # Извлекаем записи с излишками и недостатками для текущего товара
    excess = group[group['Излишки']].sort_values(by='Лишний_товар_шт', ascending=False)
    shortage = group[group['Недостаток']].sort_values(by='Недостающий_товар_шт', ascending=False)
    
    # Если нет излишков или недостатков, добавляем излишки в remaining_excess
    if excess.empty:
        continue
    if shortage.empty:
        for _, excess_row in excess.iterrows():
            if excess_row['Лишний_товар_шт'] > 0:
                remaining_excess.append({
                    'Товар': item,
                    'Код_магазина': excess_row['Код_магазина'],
                    'Код_номенклатуры': excess_row['Код_номенклатуры'],  # Добавляем Код_номенклатуры
                    'Количество_не_перемещено_шт': excess_row['Лишний_товар_шт'],
                    'Сумма_не_перемещено_тыс_руб': excess_row['Лишний_товар_шт'] * excess_row['Стоимость_за_единицу_тыс_руб']
                })
        continue
    
    # Преобразуем в списки для более быстрого доступа, включая Код_номенклатуры
    excess_list = excess[['Код_магазина', 'Код_номенклатуры', 'Лишний_товар_шт', 'Стоимость_за_единицу_тыс_руб']].to_dict('records')
    shortage_list = shortage[['Код_магазина', 'Недостающий_товар_шт']].to_dict('records')
    
    # Перераспределяем с учётом условия "минимум 3 штуки"
    for excess_entry in excess_list:
        excess_store = excess_entry['Код_магазина']
        excess_amount = excess_entry['Лишний_товар_шт']
        unit_price = excess_entry['Стоимость_за_единицу_тыс_руб']
        item_code = excess_entry['Код_номенклатуры']  # Извлекаем Код_номенклатуры
        
        for shortage_entry in shortage_list:
            if excess_amount < MIN_TRANSFER_QTY:
                break
                
            shortage_store = shortage_entry['Код_магазина']
            shortage_amount = shortage_entry['Недостающий_товар_шт']
            
            if shortage_amount >= MIN_TRANSFER_QTY:  # Перемещаем, только если можно удовлетворить минимум 3 штуки
                transfer_amount = min(excess_amount, shortage_amount)
                if transfer_amount >= MIN_TRANSFER_QTY:
                    transfer_value = transfer_amount * unit_price
                    recommendations.append({
                        'Товар': item,
                        'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                        'От_магазина': excess_store,
                        'В_магазин': shortage_store,
                        'Количество_шт': transfer_amount,
                        'Сумма_перемещения_тыс_руб': transfer_value
                    })
                    excess_amount -= transfer_amount
                    shortage_entry['Недостающий_товар_шт'] -= transfer_amount
        
        # Если после перераспределения остался излишек, добавляем его в remaining_excess
        if excess_amount > 0:
            remaining_excess.append({
                'Товар': item,
                'Код_магазина': excess_store,
                'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                'Количество_не_перемещено_шт': excess_amount,
                'Сумма_не_перемещено_тыс_руб': excess_amount * unit_price
            })

# 13. Выводим статистику по перемещённому и неперемещённому товару
recommendations_df = pd.DataFrame(recommendations)
remaining_excess_df = pd.DataFrame(remaining_excess)

# Статистика по перемещённому товару
if not recommendations_df.empty:
    total_moved_qty = recommendations_df['Количество_шт'].sum()
    total_moved_value = recommendations_df['Сумма_перемещения_тыс_руб'].sum()
else:
    total_moved_qty = 0
    total_moved_value = 0

# Статистика по неперемещённому товару
if not remaining_excess_df.empty:
    total_not_moved_qty = remaining_excess_df['Количество_не_перемещено_шт'].sum()
    total_not_moved_value = remaining_excess_df['Сумма_не_перемещено_тыс_руб'].sum()
else:
    total_not_moved_qty = 0
    total_not_moved_value = 0

print("\nСтатистика по перемещённому товару:")
print(f"Количество перемещённого товара (шт.): {total_moved_qty}")
print(f"Сумма перемещённого товара (тыс. руб.): {total_moved_value:.2f}")

print("\nСтатистика по неперемещённому товару:")
print(f"Количество неперемещённого товара (шт.): {total_not_moved_qty}")
print(f"Сумма неперемещённого товара (тыс. руб.): {total_not_moved_value:.2f}")

# 14. Проверяем, что все перемещения соответствуют условию "минимум 3 штуки"
if not recommendations_df.empty:
    invalid_transfers = recommendations_df[recommendations_df['Количество_шт'] < MIN_TRANSFER_QTY]
    if not invalid_transfers.empty:
        print("\nОшибка: Найдены перемещения с количеством меньше минимального в recommendations_df:")
        print(invalid_transfers)
    else:
        print(f"\nУспех: Все перемещения в recommendations_df соответствуют условию 'минимум {MIN_TRANSFER_QTY} штуки'.")

# 15. Выводим рекомендации по перемещению
print("\nРекомендации по перемещению товара (минимум 3 штуки):")
if not recommendations_df.empty:
    print(recommendations_df)
else:
    print("Нет возможности перераспределить товар: либо нет излишков, либо нет недостатков для одних и тех же товаров.")

# 16. Выводим неперемещённый товар
print("\nТовар, который не удалось переместить (будет распределён на указанные магазины):")
if not remaining_excess_df.empty:
    print(remaining_excess_df)
else:
    print("Весь излишний товар был перемещён.")

# 17. Распределяем ВЕСЬ неперемещённый товар пропорционально продажам на указанные магазины
# Список курортных магазинов
target_stores = [
    68, 70, 79, 80, 81, 82, 83, 85, 91, 92, 93, 97, 99, 105, 106, 112, 118, 119, 121, 124, 128, 130, 
    132, 134, 135, 140, 141, 142, 144, 145, 156, 158, 161, 162, 165, 166, 167, 168, 171, 178, 270, 292, 
    303, 321, 362, 373, 381, 392, 394, 400, 416, 429, 438, 440, 479, 487, 561, 573, 580, 585, 587, 741, 
    749, 758, 864
]

# Фильтруем данные только для указанных магазинов
target_df = df[df['Код_магазина'].isin(target_stores)].copy()

# Рассчитываем доли продаж для каждого магазина
store_sales = target_df.groupby('Код_магазина')['Кол_во_продано_шт'].sum().reset_index()
total_sales = store_sales['Кол_во_продано_шт'].sum()
store_sales['Доля'] = store_sales['Кол_во_продано_шт'] / total_sales if total_sales > 0 else 0

# Проверяем, есть ли магазины с ненулевыми продажами
if total_sales == 0:
    print("\nОшибка: Указанные магазины не имеют продаж. Распределение невозможно.")
else:
    # Сортируем магазины по доле (для распределения остатка)
    store_sales = store_sales.sort_values(by='Доля', ascending=False).reset_index(drop=True)
    
    # Распределяем ВЕСЬ неперемещённый товар с учётом условия "минимум 3 штуки"
    redistribution = []
    final_remainder = []  # Список для финального остатка (меньше 3 штук)

    for _, excess_row in remaining_excess_df.iterrows():
        item = excess_row['Товар']
        from_store = excess_row['Код_магазина']
        item_code = excess_row['Код_номенклатуры']  # Извлекаем Код_номенклатуры
        total_to_redistribute = int(excess_row['Количество_не_перемещено_шт'])  # Убедимся, что это целое число
        unit_price = excess_row['Сумма_не_перемещено_тыс_руб'] / total_to_redistribute if total_to_redistribute > 0 else 0
        
        if total_to_redistribute < MIN_TRANSFER_QTY:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        remaining_to_distribute = total_to_redistribute
        distributed_amounts = {store: 0 for store in store_sales['Код_магазина']}
        
        # Шаг 1: Распределяем пропорционально долям с учётом условия "минимум 3 штуки"
        while remaining_to_distribute >= MIN_TRANSFER_QTY:
            for _, store_row in store_sales.iterrows():
                if remaining_to_distribute < MIN_TRANSFER_QTY:
                    break
                
                to_store = store_row['Код_магазина']
                share = store_row['Доля']
                # Рассчитываем количество для распределения (округление вниз)
                amount = int(remaining_to_distribute * share)
                # Убедимся, что количество не меньше минимального
                if amount < MIN_TRANSFER_QTY:
                    amount = MIN_TRANSFER_QTY if remaining_to_distribute >= MIN_TRANSFER_QTY else 0
                
                if amount >= MIN_TRANSFER_QTY:
                    distributed_amounts[to_store] += amount
                    remaining_to_distribute -= amount
        
        # Шаг 2: Распределяем остаток (если остался) по магазинам с наибольшими долями
        while remaining_to_distribute >= MIN_TRANSFER_QTY:
            for _, store_row in store_sales.iterrows():
                if remaining_to_distribute < MIN_TRANSFER_QTY:
                    break
                to_store = store_row['Код_магазина']
                amount = MIN_TRANSFER_QTY
                distributed_amounts[to_store] += amount
                remaining_to_distribute -= amount
        
        # Шаг 3: Формируем записи для распределения
        for to_store, amount in distributed_amounts.items():
            if amount >= MIN_TRANSFER_QTY:
                value = amount * unit_price
                redistribution.append({
                    'Товар': item,
                    'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                    'Из_магазина': from_store,
                    'В_магазин': to_store,
                    'Количество_распределено_шт': amount,
                    'Сумма_распределено_тыс_руб': value
                })
            elif amount > 0:
                # Если осталось меньше MIN_TRANSFER_QTY, это не должно попасть в распределение
                remaining_to_distribute += amount
        
        # Если после всех шагов остался товар меньше MIN_TRANSFER_QTY, добавляем его в final_remainder
        if remaining_to_distribute > 0:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                'Из_магазина': from_store,
                'Количество_остаток_шт': remaining_to_distribute,
                'Сумма_остаток_тыс_руб': remaining_to_distribute * unit_price
            })

    # Создаём DataFrame для распределённого товара
    redistribution_df = pd.DataFrame(redistribution)
    # Создаём DataFrame для финального остатка
    final_remainder_df = pd.DataFrame(final_remainder)

    # 18. Проверяем, что весь товар распределён (с учётом условия)
    if not redistribution_df.empty:
        total_redistributed_qty = redistribution_df['Количество_распределено_шт'].sum()
        print(f"\nПроверка: Всего неперемещённого товара: {total_not_moved_qty} шт.")
        print(f"Всего распределено: {total_redistributed_qty} шт.")
        if total_redistributed_qty < total_not_moved_qty:
            print(f"Предупреждение: Не весь товар был распределён из-за условия 'минимум {MIN_TRANSFER_QTY} штуки'. Остаток: {total_not_moved_qty - total_redistributed_qty} шт.")
        else:
            print("Успех: Весь неперемещённый товар был распределён (с учётом условия).")
    else:
        print("\nНет неперемещённого товара для распределения.")

    # 19. Проверяем, что все перемещения соответствуют условию "минимум 3 штуки"
    if not redistribution_df.empty:
        invalid_transfers = redistribution_df[redistribution_df['Количество_распределено_шт'] < MIN_TRANSFER_QTY]
        if not invalid_transfers.empty:
            print("\nОшибка: Найдены перемещения с количеством меньше минимального в redistribution_df:")
            print(invalid_transfers)
        else:
            print(f"\nУспех: Все перемещения в redistribution_df соответствуют условию 'минимум {MIN_TRANSFER_QTY} штуки'.")

    # 20. Выводим распределение неперемещённого товара
    print("\nРаспределение неперемещённого товара по указанным магазинам (пропорционально продажам, минимум 3 штуки):")
    if not redistribution_df.empty:
        print(redistribution_df)
    else:
        print("Нет неперемещённого товара для распределения.")

    # 21. Выводим финальный остаток (меньше 3 штук)
    print("\nФинальный остаток (товар, который не удалось распределить из-за условия 'минимум 3 штуки'):")
    if not final_remainder_df.empty:
        print(final_remainder_df)
    else:
        print("Весь товар был распределён, финального остатка нет.")

    # 22. Сохраняем результаты в Excel (четыре листа)
    with pd.ExcelWriter('recommendations.xlsx', engine='openpyxl') as writer:
        recommendations_df.to_excel(writer, sheet_name='Перемещения', index=False)
        remaining_excess_df.to_excel(writer, sheet_name='Неперемещенный_товар', index=False)
        redistribution_df.to_excel(writer, sheet_name='Распределение_неперемещенного', index=False)
        final_remainder_df.to_excel(writer, sheet_name='Финальный_остаток', index=False)
    print("\nРезультаты сохранены в файл 'recommendations.xlsx' (четыре листа: 'Перемещения', 'Неперемещенный_товар', 'Распределение_неперемещенного', 'Финальный_остаток')")
    
# Выводим время выполнения
end_time = time.time()
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

# Блок 2(на 1 м этапе из всех во все.на 2 только на куротные)

In [ ]:
import pandas as pd
import numpy as np
import time

# Замеряем время выполнения
start_time = time.time()

# 1. Загружаем данные из Excel
df = pd.read_excel('Для Анализа Излишко Недостач.xlsx')

# 2. Переименовываем столбцы (10 столбцов)
df.columns = [
    'Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 
    'Сумма_продаж_тыс_руб', 'Остатки_тыс_руб', 'Ср_дневн_продажи_тыс_руб', 
    'Запас_в_днях', 'Кол_во_продано_шт', 'Торговых_дней'
]

# 3. Убедимся, что "Запас_в_днях" числовой
df['Запас_в_днях'] = pd.to_numeric(df['Запас_в_днях'], errors='coerce')

# 4. Рассчитываем стоимость одной единицы товара (в тыс. руб.)
df['Стоимость_за_единицу_тыс_руб'] = df['Остатки_тыс_руб'] / df['Остатки_шт']
df['Стоимость_за_единицу_тыс_руб'] = df['Стоимость_за_единицу_тыс_руб'].replace([np.inf, -np.inf, np.nan], 0)

# Для товаров с нулевыми остатками используем среднюю стоимость по товару
mean_price_per_item = df[df['Стоимость_за_единицу_тыс_руб'] > 0].groupby('Товар')['Стоимость_за_единицу_тыс_руб'].mean()
df = df.merge(mean_price_per_item.rename('Средняя_стоимость_за_единицу_тыс_руб'), on='Товар', how='left')
df['Стоимость_за_единицу_тыс_руб'] = np.where(
    df['Остатки_шт'] == 0,
    df['Средняя_стоимость_за_единицу_тыс_руб'].fillna(0),
    df['Стоимость_за_единицу_тыс_руб']
)

# 5. Определяем магазины с излишками и недостатками
df['Излишки'] = df['Запас_в_днях'] > 90
df['Недостаток'] = df['Запас_в_днях'] < 90

# 6. Рассчитываем средние дневные продажи в штуках
df['Ср_дневн_продажи_шт'] = df['Кол_во_продано_шт'] / df['Торговых_дней']
df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace([np.inf, -np.inf, np.nan], 0)

# Если Ср_дневн_продажи_шт = 0, используем минимальное ненулевое значение
min_sales_rate = df[df['Ср_дневн_продажи_шт'] > 0]['Ср_дневн_продажи_шт'].min()
if pd.notna(min_sales_rate):
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, min_sales_rate)
else:
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, 0.01)

# 7. Рассчитываем желаемые остатки, лишний и недостающий товар
df['Желаемые_остатки_шт'] = (df['Ср_дневн_продажи_шт'] * 90).round()
df['Лишний_товар_шт'] = np.where(df['Излишки'], (df['Остатки_шт'] - df['Желаемые_остатки_шт']).clip(lower=0), 0)
df['Недостающий_товар_шт'] = np.where(df['Недостаток'], (df['Желаемые_остатки_шт'] - df['Остатки_шт']).clip(lower=0), 0)

# 8. Перераспределяем товар с учётом условия "минимум 3 штуки" и отслеживаем неперемещённый товар
recommendations = []
remaining_excess = []
MIN_TRANSFER_QTY = 3  # Минимальное количество для перемещения

# Список курортных магазинов (определяем заранее, чтобы использовать в шаге 8)
target_stores = [
    68, 70, 79, 80, 81, 82, 83, 85, 91, 92, 93, 97, 99, 105, 106, 112, 118, 119, 121, 124, 128, 130, 
    132, 134, 135, 140, 141, 142, 144, 145, 156, 158, 161, 162, 165, 166, 167, 168, 171, 178, 270, 292, 
    303, 321, 362, 373, 381, 392, 394, 400, 416, 429, 438, 440, 479, 487, 561, 573, 580, 585, 587, 741, 
    749, 758, 864
]

# Группируем по товару
grouped = df.groupby('Товар')

for item, group in grouped:
    # Извлекаем записи с излишками и недостатками для текущего товара
    excess = group[group['Излишки']].sort_values(by='Лишний_товар_шт', ascending=False)
    shortage = group[group['Недостаток']].sort_values(by='Недостающий_товар_шт', ascending=False)
    
    # Если нет излишков или недостатков, добавляем излишки в remaining_excess
    if excess.empty:
        continue
    if shortage.empty:
        for _, excess_row in excess.iterrows():
            # Добавляем излишки в remaining_excess только для магазинов, НЕ входящих в target_stores
            if excess_row['Лишний_товар_шт'] > 0 and excess_row['Код_магазина'] not in target_stores:
                remaining_excess.append({
                    'Товар': item,
                    'Код_магазина': excess_row['Код_магазина'],
                    'Код_номенклатуры': excess_row['Код_номенклатуры'],  # Добавляем Код_номенклатуры
                    'Количество_не_перемещено_шт': excess_row['Лишний_товар_шт'],
                    'Сумма_не_перемещено_тыс_руб': excess_row['Лишний_товар_шт'] * excess_row['Стоимость_за_единицу_тыс_руб']
                })
        continue
    
    # Преобразуем в списки для более быстрого доступа, включая Код_номенклатуры
    excess_list = excess[['Код_магазина', 'Код_номенклатуры', 'Лишний_товар_шт', 'Стоимость_за_единицу_тыс_руб']].to_dict('records')
    shortage_list = shortage[['Код_магазина', 'Недостающий_товар_шт']].to_dict('records')
    
    # Перераспределяем с учётом условия "минимум 3 штуки"
    for excess_entry in excess_list:
        excess_store = excess_entry['Код_магазина']
        excess_amount = excess_entry['Лишний_товар_шт']
        unit_price = excess_entry['Стоимость_за_единицу_тыс_руб']
        item_code = excess_entry['Код_номенклатуры']  # Извлекаем Код_номенклатуры
        
        for shortage_entry in shortage_list:
            if excess_amount < MIN_TRANSFER_QTY:
                break
                
            shortage_store = shortage_entry['Код_магазина']
            shortage_amount = shortage_entry['Недостающий_товар_шт']
            
            if shortage_amount >= MIN_TRANSFER_QTY:  # Перемещаем, только если можно удовлетворить минимум 3 штуки
                transfer_amount = min(excess_amount, shortage_amount)
                if transfer_amount >= MIN_TRANSFER_QTY:
                    transfer_value = transfer_amount * unit_price
                    recommendations.append({
                        'Товар': item,
                        'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                        'От_магазина': excess_store,
                        'В_магазин': shortage_store,
                        'Количество_шт': transfer_amount,
                        'Сумма_перемещения_тыс_руб': transfer_value
                    })
                    excess_amount -= transfer_amount
                    shortage_entry['Недостающий_товар_шт'] -= transfer_amount
        
        # Если после перераспределения остался излишек, добавляем его в remaining_excess, но только для магазинов, НЕ входящих в target_stores
        if excess_amount > 0 and excess_store not in target_stores:
            remaining_excess.append({
                'Товар': item,
                'Код_магазина': excess_store,
                'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                'Количество_не_перемещено_шт': excess_amount,
                'Сумма_не_перемещено_тыс_руб': excess_amount * unit_price
            })

# 9. Выводим статистику по перемещённому и неперемещённому товару
recommendations_df = pd.DataFrame(recommendations)
remaining_excess_df = pd.DataFrame(remaining_excess)

# Статистика по перемещённому товару
if not recommendations_df.empty:
    total_moved_qty = recommendations_df['Количество_шт'].sum()
    total_moved_value = recommendations_df['Сумма_перемещения_тыс_руб'].sum()
else:
    total_moved_qty = 0
    total_moved_value = 0

# Статистика по неперемещённому товару
if not remaining_excess_df.empty:
    total_not_moved_qty = remaining_excess_df['Количество_не_перемещено_шт'].sum()
    total_not_moved_value = remaining_excess_df['Сумма_не_перемещено_тыс_руб'].sum()
else:
    total_not_moved_qty = 0
    total_not_moved_value = 0

print("\nСтатистика по перемещённому товару:")
print(f"Количество перемещённого товара (шт.): {total_moved_qty}")
print(f"Сумма перемещённого товара (тыс. руб.): {total_moved_value:.2f}")

print("\nСтатистика по неперемещённому товару:")
print(f"Количество неперемещённого товара (шт.): {total_not_moved_qty}")
print(f"Сумма неперемещённого товара (тыс. руб.): {total_not_moved_value:.2f}")

# 10. Распределяем ВЕСЬ неперемещённый товар на указанные магазины по потребности (на основе Запас_в_днях и Недостающий_товар_шт)
# Фильтруем данные только для указанных магазинов
target_df = df[df['Код_магазина'].isin(target_stores)].copy()

# Проверяем, есть ли магазины для распределения
if target_df.empty:
    raise ValueError("Список целевых магазинов пуст. Распределение невозможно.")
else:
    # Распределяем ВЕСЬ неперемещённый товар с учётом потребности (Запас_в_днях и Недостающий_товар_шт)
    redistribution = []
    final_remainder = []  # Список для финального остатка (меньше 3 штук)
    MIN_TRANSFER_QTY = 3  # Минимальное количество для перемещения
    MAX_PER_STORE = 24  # Максимум 18 единиц одного SKU на магазин (по вашему запросу)

    # Словарь для отслеживания общего количества товара одного SKU в каждом магазине
    total_per_store_per_sku = {}

    for _, excess_row in remaining_excess_df.iterrows():
        item = excess_row['Товар']
        from_store = excess_row['Код_магазина']
        item_code = excess_row['Код_номенклатуры']  # Извлекаем Код_номенклатуры
        total_to_redistribute = int(excess_row['Количество_не_перемещено_шт'])  # Убедимся, что это целое число
        unit_price = excess_row['Сумма_не_перемещено_тыс_руб'] / total_to_redistribute if total_to_redistribute > 0 else 0
        
        if total_to_redistribute < MIN_TRANSFER_QTY:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        # Фильтруем target_df по текущему товару
        item_target_df = target_df[target_df['Товар'] == item].copy()
        
        # Если для данного товара нет записей в target_df, добавляем в final_remainder
        if item_target_df.empty:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        # Сортируем магазины по Запас_в_днях (по возрастанию) и учитываем Недостающий_товар_шт
        store_needs = item_target_df.groupby('Код_магазина').agg({
            'Запас_в_днях': 'mean',
            'Недостающий_товар_шт': 'sum',
            'Ср_дневн_продажи_шт': 'mean'
        }).reset_index()
        store_needs = store_needs.sort_values(by=['Запас_в_днях', 'Недостающий_товар_шт'], ascending=[True, False]).reset_index(drop=True)
        
        remaining_to_distribute = total_to_redistribute
        distributed_amounts = {store: 0 for store in store_needs['Код_магазина']}
        
        # Преобразуем store_needs в список для более быстрого доступа
        store_list = [
            (row['Код_магазина'], row['Запас_в_днях'], row['Недостающий_товар_шт'])
            for _, row in store_needs.iterrows()
        ]
        
        # Инициализируем общее количество товара для данного SKU
        if item_code not in total_per_store_per_sku:
            total_per_store_per_sku[item_code] = {store: 0 for store in target_stores}
        
        # Шаг 1: Распределяем товар по магазинам с наименьшим запасом, но не больше их потребности и не больше 18 единиц
        store_index = 0
        num_stores = len(store_list)
        while remaining_to_distribute >= MIN_TRANSFER_QTY and store_index < num_stores:
            store, stock_days, shortage = store_list[store_index]
            
            # Проверяем общее количество товара данного SKU в магазине
            current_total_in_store = total_per_store_per_sku[item_code][store]
            if current_total_in_store >= MAX_PER_STORE:
                store_index += 1
                continue
            
            # Пропускаем, если потребность уже удовлетворена
            remaining_shortage = max(0, shortage - distributed_amounts[store])
            if remaining_shortage <= 0:
                store_index += 1
                continue
            
            # Учитываем максимальное количество, которое можно переместить в этот магазин
            max_allowed_to_store = MAX_PER_STORE - current_total_in_store
            if max_allowed_to_store < MIN_TRANSFER_QTY:
                store_index += 1
                continue
            
            amount = min(remaining_to_distribute, remaining_shortage, max_allowed_to_store)
            if amount < MIN_TRANSFER_QTY:
                store_index += 1
                continue
            
            distributed_amounts[store] += amount
            total_per_store_per_sku[item_code][store] += amount
            remaining_to_distribute -= amount
            if remaining_shortage <= amount or total_per_store_per_sku[item_code][store] >= MAX_PER_STORE:
                store_index += 1  # Переходим к следующему магазину
        
        # Шаг 2: Если остался товар, распределяем его по остальным магазинам с наименьшим Запас_в_днях, но не больше 18 единиц на магазин
        if remaining_to_distribute >= MIN_TRANSFER_QTY:
            # Создаём список магазинов, которые ещё могут принимать товар
            remaining_stores = [
                (store, stock_days)
                for store, stock_days, _ in store_list
                if total_per_store_per_sku[item_code][store] < MAX_PER_STORE
            ]
            remaining_stores.sort(key=lambda x: x[1])  # Сортировка по Запас_в_днях (возрастание)
            
            # Распределяем товар по магазинам, строго соблюдая лимит в 18 единиц
            store_index = 0
            num_stores = len(remaining_stores)
            while remaining_to_distribute >= MIN_TRANSFER_QTY and num_stores > 0:
                store, _ = remaining_stores[store_index]
                current_total_in_store = total_per_store_per_sku[item_code][store]
                
                if current_total_in_store >= MAX_PER_STORE:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                max_allowed_to_store = MAX_PER_STORE - current_total_in_store
                if max_allowed_to_store < MIN_TRANSFER_QTY:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                amount = min(remaining_to_distribute, max_allowed_to_store)
                if amount < MIN_TRANSFER_QTY:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                distributed_amounts[store] += amount
                total_per_store_per_sku[item_code][store] += amount
                remaining_to_distribute -= amount
                
                if total_per_store_per_sku[item_code][store] >= MAX_PER_STORE:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                else:
                    store_index = (store_index + 1) % num_stores
        
        # Шаг 3: Формируем записи для распределения
        for to_store, amount in distributed_amounts.items():
            if amount >= MIN_TRANSFER_QTY:
                value = amount * unit_price
                redistribution.append({
                    'Товар': item,
                    'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                    'Из_магазина': from_store,
                    'В_магазин': to_store,
                    'Количество_распределено_шт': amount,
                    'Сумма_распределено_тыс_руб': value
                })
            elif amount > 0:
                remaining_to_distribute += amount
        
        # Если после всех шагов остался товар меньше MIN_TRANSFER_QTY, добавляем его в final_remainder
        if remaining_to_distribute > 0:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,  # Добавляем Код_номенклатуры
                'Из_магазина': from_store,
                'Количество_остаток_шт': remaining_to_distribute,
                'Сумма_остаток_тыс_руб': remaining_to_distribute * unit_price
            })

    # Создаём DataFrame для распределённого товара
    redistribution_df = pd.DataFrame(redistribution)
    # Создаём DataFrame для финального остатка
    final_remainder_df = pd.DataFrame(final_remainder)

    # Дополнительная проверка: убедимся, что ни один магазин не получил более 18 единиц
    if not redistribution_df.empty:
        store_totals = redistribution_df.groupby(['Код_номенклатуры', 'В_магазин'])['Количество_распределено_шт'].sum().reset_index()
        over_limit = store_totals[store_totals['Количество_распределено_шт'] > MAX_PER_STORE]
        if not over_limit.empty:
            raise ValueError(f"Ошибка: Некоторые магазины получили более {MAX_PER_STORE} единиц товара:\n{over_limit}")

    # 11. Сохраняем результаты в Excel (четыре листа)
    with pd.ExcelWriter('recommendations2.xlsx', engine='openpyxl') as writer:
        recommendations_df.to_excel(writer, sheet_name='Перемещения', index=False)
        remaining_excess_df.to_excel(writer, sheet_name='Неперемещенный_товар', index=False)
        redistribution_df.to_excel(writer, sheet_name='Распределение_неперемещенного', index=False)
        final_remainder_df.to_excel(writer, sheet_name='Финальный_остаток', index=False)

# Выводим время выполнения
end_time = time.time()
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

# Блок 3(на 1 и 2 этапах только на курортные)

In [ ]:
import pandas as pd
import numpy as np
import time

# Замеряем время выполнения
start_time = time.time()

# 1. Загружаем данные из Excel
df = pd.read_excel('Для Анализа Излишко Недостач3.xlsx')

# 2. Переименовываем столбцы (10 столбцов + новый столбец Статус)
df.columns = [
    'Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 
    'Сумма_продаж_тыс_руб', 'Остатки_тыс_руб', 'Ср_дневн_продажи_тыс_руб', 
    'Запас_в_днях', 'Кол_во_продано_шт', 'Торговых_дней', 'Статус'
]

# 3. Убедимся, что "Запас_в_днях" и "Статус" числовые
df['Запас_в_днях'] = pd.to_numeric(df['Запас_в_днях'], errors='coerce')
df['Статус'] = pd.to_numeric(df['Статус'], errors='coerce')

# 4. Рассчитываем стоимость одной единицы товара (в тыс. руб.)
df['Стоимость_за_единицу_тыс_руб'] = df['Остатки_тыс_руб'] / df['Остатки_шт']
df['Стоимость_за_единицу_тыс_руб'] = df['Стоимость_за_единицу_тыс_руб'].replace([np.inf, -np.inf, np.nan], 0)

# Для товаров с нулевыми остатками используем среднюю стоимость по товару
mean_price_per_item = df[df['Стоимость_за_единицу_тыс_руб'] > 0].groupby('Товар')['Стоимость_за_единицу_тыс_руб'].mean()
df = df.merge(mean_price_per_item.rename('Средняя_стоимость_за_единицу_тыс_руб'), on='Товар', how='left')
df['Стоимость_за_единицу_тыс_руб'] = np.where(
    df['Остатки_шт'] == 0,
    df['Средняя_стоимость_за_единицу_тыс_руб'].fillna(0),
    df['Стоимость_за_единицу_тыс_руб']
)

# 5. Определяем магазины с излишками и недостатками
df['Излишки'] = df['Запас_в_днях'] > 90
df['Недостаток'] = df['Запас_в_днях'] < 90

# 6. Рассчитываем средние дневные продажи в штуках
df['Ср_дневн_продажи_шт'] = df['Кол_во_продано_шт'] / df['Торговых_дней']
df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace([np.inf, -np.inf, np.nan], 0)

# Если Ср_дневн_продажи_шт = 0, используем минимальное ненулевое значение
min_sales_rate = df[df['Ср_дневн_продажи_шт'] > 0]['Ср_дневн_продажи_шт'].min()
if pd.notna(min_sales_rate):
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, min_sales_rate)
else:
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, 0.01)

# 7. Рассчитываем желаемые остатки, лишний и недостающий товар
df['Желаемые_остатки_шт'] = (df['Ср_дневн_продажи_шт'] * 90).round()
df['Лишний_товар_шт'] = np.where(df['Излишки'], (df['Остатки_шт'] - df['Желаемые_остатки_шт']).clip(lower=0), 0)
df['Недостающий_товар_шт'] = np.where(df['Недостаток'], (df['Желаемые_остатки_шт'] - df['Остатки_шт']).clip(lower=0), 0)

# 8. Перераспределяем товар с учётом условия "минимум 3 штуки" и отслеживаем неперемещённый товар
recommendations = []
remaining_excess = []
MIN_TRANSFER_QTY = 3  # Минимальное количество для перемещения
MIN_STOCK_ACTIVE = 6  # Минимальный остаток для товаров со Статус = 1

# Список курортных магазинов
target_stores = [
    68, 70, 79, 80, 81, 82, 83, 85, 91, 92, 93, 97, 99, 105, 106, 112, 118, 119, 121, 124, 128, 130, 
    132, 134, 135, 140, 141, 142, 144, 145, 156, 158, 161, 162, 165, 166, 167, 168, 171, 178, 270, 292, 
    303, 321, 362, 373, 381, 392, 394, 400, 416, 429, 438, 440, 479, 487, 561, 573, 580, 585, 587, 741, 
    749, 758, 864, 74
]

# Группируем по товару
grouped = df.groupby('Товар')

for item, group in grouped:
    # Извлекаем записи с излишками и недостатками для текущего товара
    # Исключаем магазины из target_stores из списка излишков
    excess = group[(group['Излишки']) & (~group['Код_магазина'].isin(target_stores))].sort_values(by='Лишний_товар_шт', ascending=False)
    shortage = group[group['Недостаток']].sort_values(by='Недостающий_товар_шт', ascending=False)
    
    # Если нет излишков или недостатков, добавляем излишки в remaining_excess
    if excess.empty:
        continue
    if shortage.empty:
        for _, excess_row in excess.iterrows():
            if excess_row['Лишний_товар_шт'] > 0 and excess_row['Код_магазина'] not in target_stores:
                remaining_excess.append({
                    'Товар': item,
                    'Код_магазина': excess_row['Код_магазина'],
                    'Код_номенклатуры': excess_row['Код_номенклатуры'],
                    'Количество_не_перемещено_шт': excess_row['Лишний_товар_шт'],
                    'Сумма_не_перемещено_тыс_руб': excess_row['Лишний_товар_шт'] * excess_row['Стоимость_за_единицу_тыс_руб'],
                    'Статус': excess_row['Статус']  # Добавляем статус
                })
        continue
    
    # Преобразуем в списки для более быстрого доступа, включая Код_номенклатуры, Остатки_шт и Статус
    excess_list = excess[['Код_магазина', 'Код_номенклатуры', 'Лишний_товар_шт', 'Стоимость_за_единицу_тыс_руб', 'Остатки_шт', 'Статус']].to_dict('records')
    shortage_list = shortage[['Код_магазина', 'Недостающий_товар_шт']].to_dict('records')
    
    # Перераспределяем с учётом условия "минимум 3 штуки"
    for excess_entry in excess_list:
        excess_store = excess_entry['Код_магазина']
        excess_amount = excess_entry['Лишний_товар_шт']
        unit_price = excess_entry['Стоимость_за_единицу_тыс_руб']
        item_code = excess_entry['Код_номенклатуры']
        initial_stock = excess_entry['Остатки_шт']
        status = excess_entry['Статус']  # Статус товара
        
        for shortage_entry in shortage_list:
            if excess_amount < MIN_TRANSFER_QTY:
                break
                
            shortage_store = shortage_entry['Код_магазина']
            shortage_amount = shortage_entry['Недостающий_товар_шт']
            
            if shortage_amount >= MIN_TRANSFER_QTY:
                transfer_amount = min(excess_amount, shortage_amount)
                if transfer_amount >= MIN_TRANSFER_QTY:
                    # Проверяем остаток после перемещения для товаров со Статус = 1
                    remaining_stock = initial_stock - transfer_amount
                    if status == 1 and remaining_stock < MIN_STOCK_ACTIVE:
                        # Корректируем количество, чтобы остаток был >= 6
                        max_transfer = initial_stock - MIN_STOCK_ACTIVE
                        if max_transfer < MIN_TRANSFER_QTY:
                            break  # Пропускаем, если нельзя переместить минимум 3 единицы
                        transfer_amount = min(transfer_amount, max_transfer)
                        remaining_stock = initial_stock - transfer_amount
                    
                    transfer_value = transfer_amount * unit_price
                    recommendations.append({
                        'Товар': item,
                        'Код_номенклатуры': item_code,
                        'От_магазина': excess_store,
                        'В_магазин': shortage_store,
                        'Количество_шт': transfer_amount,
                        'Сумма_перемещения_тыс_руб': transfer_value,
                        'Остаток_после_перемещения_шт': remaining_stock
                    })
                    excess_amount -= transfer_amount
                    initial_stock -= transfer_amount
                    shortage_entry['Недостающий_товар_шт'] -= transfer_amount
        
        # Если после перераспределения остался излишек, добавляем его в remaining_excess
        if excess_amount > 0 and excess_store not in target_stores:
            remaining_excess.append({
                'Товар': item,
                'Код_магазина': excess_store,
                'Код_номенклатуры': item_code,
                'Количество_не_перемещено_шт': excess_amount,
                'Сумма_не_перемещено_тыс_руб': excess_amount * unit_price,
                'Статус': status  # Добавляем статус
            })

# 9. Выводим статистику по перемещённому и неперемещённому товару
recommendations_df = pd.DataFrame(recommendations)
remaining_excess_df = pd.DataFrame(remaining_excess)

# Статистика по перемещённому товару
if not recommendations_df.empty:
    total_moved_qty = recommendations_df['Количество_шт'].sum()
    total_moved_value = recommendations_df['Сумма_перемещения_тыс_руб'].sum()
else:
    total_moved_qty = 0
    total_moved_value = 0

# Статистика по неперемещённому товару
if not remaining_excess_df.empty:
    total_not_moved_qty = remaining_excess_df['Количество_не_перемещено_шт'].sum()
    total_not_moved_value = remaining_excess_df['Сумма_не_перемещено_тыс_руб'].sum()
else:
    total_not_moved_qty = 0
    total_not_moved_value = 0

print("\nСтатистика по перемещённому товару:")
print(f"Количество перемещённого товара (шт.): {total_moved_qty}")
print(f"Сумма перемещённого товара (тыс. руб.): {total_moved_value:.2f}")

print("\nСтатистика по неперемещённому товару:")
print(f"Количество неперемещённого товара (шт.): {total_not_moved_qty}")
print(f"Сумма неперемещённого товара (тыс. руб.): {total_not_moved_value:.2f}")

# 10. Распределяем ВЕСЬ неперемещённый товар на указанные магазины по потребности
# Фильтруем данные только для указанных магазинов
target_df = df[df['Код_магазина'].isin(target_stores)].copy()

# Проверяем, есть ли магазины для распределения
if target_df.empty:
    raise ValueError("Список целевых магазинов пуст. Распределение невозможно.")
else:
    # Распределяем ВЕСЬ неперемещённый товар с учётом потребности
    redistribution = []
    final_remainder = []
    MIN_TRANSFER_QTY = 3
    MAX_PER_STORE = 30
    MIN_STOCK_ACTIVE = 6  # Минимальный остаток для товаров со Статус = 1

    # Словарь для отслеживания общего количества товара одного SKU в каждом магазине
    total_per_store_per_sku = {}

    # Словарь для отслеживания остатка в магазинах-источниках
    remaining_stock_per_store = {}

    for _, excess_row in remaining_excess_df.iterrows():
        item = excess_row['Товар']
        from_store = excess_row['Код_магазина']
        item_code = excess_row['Код_номенклатуры']
        total_to_redistribute = int(excess_row['Количество_не_перемещено_шт'])
        unit_price = excess_row['Сумма_не_перемещено_тыс_руб'] / total_to_redistribute if total_to_redistribute > 0 else 0
        status = excess_row['Статус']  # Статус товара
        
        # Инициализируем остаток для текущего магазина-источника
        if (item_code, from_store) not in remaining_stock_per_store:
            remaining_stock_per_store[(item_code, from_store)] = total_to_redistribute
        
        if total_to_redistribute < MIN_TRANSFER_QTY:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        # Фильтруем target_df по текущему товару
        item_target_df = target_df[target_df['Товар'] == item].copy()
        
        if item_target_df.empty:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        # Сортируем магазины по Запас_в_днях
        store_needs = item_target_df.groupby('Код_магазина').agg({
            'Запас_в_днях': 'mean',
            'Недостающий_товар_шт': 'sum',
            'Ср_дневн_продажи_шт': 'mean'
        }).reset_index()
        store_needs = store_needs.sort_values(by=['Запас_в_днях', 'Недостающий_товар_шт'], ascending=[True, False]).reset_index(drop=True)
        
        remaining_to_distribute = total_to_redistribute
        distributed_amounts = {store: 0 for store in store_needs['Код_магазина']}
        
        store_list = [
            (row['Код_магазина'], row['Запас_в_днях'], row['Недостающий_товар_шт'])
            for _, row in store_needs.iterrows()
        ]
        
        if item_code not in total_per_store_per_sku:
            total_per_store_per_sku[item_code] = {store: 0 for store in target_stores}
        
        # Шаг 1: Распределяем по магазинам с наименьшим запасом
        store_index = 0
        num_stores = len(store_list)
        while remaining_to_distribute >= MIN_TRANSFER_QTY and store_index < num_stores:
            store, stock_days, shortage = store_list[store_index]
            
            current_total_in_store = total_per_store_per_sku[item_code][store]
            if current_total_in_store >= MAX_PER_STORE:
                store_index += 1
                continue
            
            remaining_shortage = max(0, shortage - distributed_amounts[store])
            if remaining_shortage <= 0:
                store_index += 1
                continue
            
            max_allowed_to_store = MAX_PER_STORE - current_total_in_store
            if max_allowed_to_store < MIN_TRANSFER_QTY:
                store_index += 1
                continue
            
            amount = min(remaining_to_distribute, remaining_shortage, max_allowed_to_store)
            if amount < MIN_TRANSFER_QTY:
                store_index += 1
                continue
            
            # Проверяем остаток после распределения для товаров со Статус = 1
            remaining_stock = remaining_stock_per_store[(item_code, from_store)] - amount
            if status == 1 and remaining_stock < MIN_STOCK_ACTIVE:
                # Корректируем количество, чтобы остаток был >= 6
                max_distribute = remaining_stock_per_store[(item_code, from_store)] - MIN_STOCK_ACTIVE
                if max_distribute < MIN_TRANSFER_QTY:
                    store_index += 1
                    continue
                amount = min(amount, max_distribute)
                remaining_stock = remaining_stock_per_store[(item_code, from_store)] - amount
            
            distributed_amounts[store] += amount
            total_per_store_per_sku[item_code][store] += amount
            remaining_to_distribute -= amount
            if remaining_shortage <= amount or total_per_store_per_sku[item_code][store] >= MAX_PER_STORE:
                store_index += 1
        
        # Шаг 2: Распределяем остатки по магазинам с наименьшим запасом
        if remaining_to_distribute >= MIN_TRANSFER_QTY:
            remaining_stores = [
                (store, stock_days)
                for store, stock_days, _ in store_list
                if total_per_store_per_sku[item_code][store] < MAX_PER_STORE
            ]
            remaining_stores.sort(key=lambda x: x[1])
            
            store_index = 0
            num_stores = len(remaining_stores)
            while remaining_to_distribute >= MIN_TRANSFER_QTY and num_stores > 0:
                store, _ = remaining_stores[store_index]
                current_total_in_store = total_per_store_per_sku[item_code][store]
                
                if current_total_in_store >= MAX_PER_STORE:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                max_allowed_to_store = MAX_PER_STORE - current_total_in_store
                if max_allowed_to_store < MIN_TRANSFER_QTY:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                amount = min(remaining_to_distribute, max_allowed_to_store)
                if amount < MIN_TRANSFER_QTY:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                # Проверяем остаток после распределения для товаров со Статус = 1
                remaining_stock = remaining_stock_per_store[(item_code, from_store)] - amount
                if status == 1 and remaining_stock < MIN_STOCK_ACTIVE:
                    max_distribute = remaining_stock_per_store[(item_code, from_store)] - MIN_STOCK_ACTIVE
                    if max_distribute < MIN_TRANSFER_QTY:
                        remaining_stores.pop(store_index)
                        num_stores -= 1
                        if num_stores > 0:
                            store_index = store_index % num_stores
                        continue
                    amount = min(amount, max_distribute)
                    remaining_stock = remaining_stock_per_store[(item_code, from_store)] - amount
                
                distributed_amounts[store] += amount
                total_per_store_per_sku[item_code][store] += amount
                remaining_to_distribute -= amount
                
                if total_per_store_per_sku[item_code][store] >= MAX_PER_STORE:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                else:
                    store_index = (store_index + 1) % num_stores
        
        # Шаг 3: Формируем записи для распределения
        for to_store, amount in distributed_amounts.items():
            if amount >= MIN_TRANSFER_QTY:
                value = amount * unit_price
                # Обновляем остаток в магазине-источнике
                remaining_stock_per_store[(item_code, from_store)] -= amount
                remaining_stock = remaining_stock_per_store[(item_code, from_store)]
                redistribution.append({
                    'Товар': item,
                    'Код_номенклатуры': item_code,
                    'Из_магазина': from_store,
                    'В_магазин': to_store,
                    'Количество_распределено_шт': amount,
                    'Сумма_распределено_тыс_руб': value,
                    'Остаток_после_распределения_шт': remaining_stock
                })
            elif amount > 0:
                remaining_to_distribute += amount
        
        # Если после всех шагов остался товар меньше MIN_TRANSFER_QTY
        if remaining_to_distribute > 0:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,
                'Из_магазина': from_store,
                'Количество_остаток_шт': remaining_to_distribute,
                'Сумма_остаток_тыс_руб': remaining_to_distribute * unit_price
            })

    # Создаём DataFrame для распределённого товара
    redistribution_df = pd.DataFrame(redistribution)
    # Создаём DataFrame для финального остатка
    final_remainder_df = pd.DataFrame(final_remainder)

    # Проверяем, что ни один магазин не получил более 18 единиц
    if not redistribution_df.empty:
        store_totals = redistribution_df.groupby(['Код_номенклатуры', 'В_магазин'])['Количество_распределено_шт'].sum().reset_index()
        over_limit = store_totals[store_totals['Количество_распределено_шт'] > MAX_PER_STORE]
        if not over_limit.empty:
            raise ValueError(f"Ошибка: Некоторые магазины получили более {MAX_PER_STORE} единиц товара:\n{over_limit}")

    # 11. Сохраняем результаты в Excel
    with pd.ExcelWriter('recommendations3.xlsx', engine='openpyxl') as writer:
        recommendations_df.to_excel(writer, sheet_name='Перемещения', index=False)
        remaining_excess_df.to_excel(writer, sheet_name='Неперемещенный_товар', index=False)
        redistribution_df.to_excel(writer, sheet_name='Распределение_неперемещенного', index=False)
        final_remainder_df.to_excel(writer, sheet_name='Финальный_остаток', index=False)

# Выводим время выполнения
end_time = time.time()
print(f"Время выполнения: {end_time - start_time:.2f} секунд")

# Блок 4 (объединение этапов)

In [ ]:
import pandas as pd
import numpy as np
import time

# Замеряем время выполнения
start_time = time.time()

# 1. Загружаем данные из Excel
df = pd.read_excel('Для Анализа Излишко Недостач3.xlsx')

# 2. Переименовываем столбцы (10 столбцов + новый столбец Статус)
df.columns = [
    'Код_магазина', 'Товар', 'Код_номенклатуры', 'Остатки_шт', 
    'Сумма_продаж_тыс_руб', 'Остатки_тыс_руб', 'Ср_дневн_продажи_тыс_руб', 
    'Запас_в_днях', 'Кол_во_продано_шт', 'Торговых_дней', 'Статус'
]

# 3. Убедимся, что "Запас_в_днях" и "Статус" числовые
df['Запас_в_днях'] = pd.to_numeric(df['Запас_в_днях'], errors='coerce')
df['Статус'] = pd.to_numeric(df['Статус'], errors='coerce')

# 4. Рассчитываем стоимость одной единицы товара (в тыс. руб.)
df['Стоимость_за_единицу_тыс_руб'] = df['Остатки_тыс_руб'] / df['Остатки_шт']
df['Стоимость_за_единицу_тыс_руб'] = df['Стоимость_за_единицу_тыс_руб'].replace([np.inf, -np.inf, np.nan], 0)

# Для товаров с нулевыми остатками используем среднюю стоимость по товару
mean_price_per_item = df[df['Стоимость_за_единицу_тыс_руб'] > 0].groupby('Товар')['Стоимость_за_единицу_тыс_руб'].mean()
df = df.merge(mean_price_per_item.rename('Средняя_стоимость_за_единицу_тыс_руб'), on='Товар', how='left')
df['Стоимость_за_единицу_тыс_руб'] = np.where(
    df['Остатки_шт'] == 0,
    df['Средняя_стоимость_за_единицу_тыс_руб'].fillna(0),
    df['Стоимость_за_единицу_тыс_руб']
)

# 5. Определяем магазины с излишками и недостатками
df['Излишки'] = df['Запас_в_днях'] > 90
df['Недостаток'] = df['Запас_в_днях'] < 90

# 6. Рассчитываем средние дневные продажи в штуках
df['Ср_дневн_продажи_шт'] = df['Кол_во_продано_шт'] / df['Торговых_дней']
df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace([np.inf, -np.inf, np.nan], 0)

# Если Ср_дневн_продажи_шт = 0, используем минимальное ненулевое значение
min_sales_rate = df[df['Ср_дневн_продажи_шт'] > 0]['Ср_дневн_продажи_шт'].min()
if pd.notna(min_sales_rate):
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, min_sales_rate)
else:
    df['Ср_дневн_продажи_шт'] = df['Ср_дневн_продажи_шт'].replace(0, 0.01)

# 7. Рассчитываем желаемые остатки, лишний и недостающий товар
df['Желаемые_остатки_шт'] = (df['Ср_дневн_продажи_шт'] * 90).round()
df['Лишний_товар_шт'] = np.where(df['Излишки'], (df['Остатки_шт'] - df['Желаемые_остатки_шт']).clip(lower=0), 0)
df['Недостающий_товар_шт'] = np.where(df['Недостаток'], (df['Желаемые_остатки_шт'] - df['Остатки_шт']).clip(lower=0), 0)

# 8. Перераспределяем товар с учётом условия "минимум 3 штуки" и отслеживаем неперемещённый товар
recommendations = []
remaining_excess = []
MIN_TRANSFER_QTY = 3  # Минимальное количество для перемещения
MIN_STOCK_ACTIVE = 6  # Минимальный остаток для товаров со Статус = 1

# Список курортных магазинов
target_stores = [
    68, 70, 79, 80, 81, 82, 83, 85, 91, 92, 93, 97, 99, 105, 106, 112, 118, 119, 121, 124, 128, 130, 
    132, 134, 135, 140, 141, 142, 144, 145, 156, 158, 161, 162, 165, 166, 167, 168, 171, 178, 270, 292, 
    303, 321, 362, 373, 381, 392, 394, 400, 416, 429, 438, 440, 479, 487, 561, 573, 580, 585, 587, 741, 
    749, 758, 864, 74
]

# Словарь для отслеживания общего перемещённого количества по товару и магазину-источнику
total_transferred_per_item_store = {}

# Группируем по товару
grouped = df.groupby('Товар')

for item, group in grouped:
    # Извлекаем записи с излишками и недостатками для текущего товара
    # Исключаем магазины из target_stores из списка излишков
    excess = group[(group['Излишки']) & (~group['Код_магазина'].isin(target_stores))].sort_values(by='Лишний_товар_шт', ascending=False)
    shortage = group[group['Недостаток']].sort_values(by='Недостающий_товар_шт', ascending=False)
    
    # Если нет излишков или недостатков, добавляем излишки в remaining_excess
    if excess.empty:
        continue
    if shortage.empty:
        for _, excess_row in excess.iterrows():
            if excess_row['Лишний_товар_шт'] > 0 and excess_row['Код_магазина'] not in target_stores:
                remaining_excess.append({
                    'Товар': item,
                    'Код_магазина': excess_row['Код_магазина'],
                    'Код_номенклатуры': excess_row['Код_номенклатуры'],
                    'Количество_не_перемещено_шт': excess_row['Лишний_товар_шт'],
                    'Сумма_не_перемещено_тыс_руб': excess_row['Лишний_товар_шт'] * excess_row['Стоимость_за_единицу_тыс_руб'],
                    'Статус': excess_row['Статус']
                })
        continue
    
    # Преобразуем в списки для более быстрого доступа, включая Код_номенклатуры, Остатки_шт и Статус
    excess_list = excess[['Код_магазина', 'Код_номенклатуры', 'Лишний_товар_шт', 'Стоимость_за_единицу_тыс_руб', 'Остатки_шт', 'Статус']].to_dict('records')
    shortage_list = shortage[['Код_магазина', 'Недостающий_товар_шт']].to_dict('records')
    
    # Перераспределяем с учётом условия "минимум 3 штуки"
    for excess_entry in excess_list:
        excess_store = excess_entry['Код_магазина']
        excess_amount = excess_entry['Лишний_товар_шт']
        unit_price = excess_entry['Стоимость_за_единицу_тыс_руб']
        item_code = excess_entry['Код_номенклатуры']
        initial_stock = excess_entry['Остатки_шт']
        status = excess_entry['Статус']
        
        # Инициализируем счётчик перемещённого товара для текущего магазина и товара
        key = (item_code, excess_store)
        if key not in total_transferred_per_item_store:
            total_transferred_per_item_store[key] = 0
        
        for shortage_entry in shortage_list:
            if excess_amount < MIN_TRANSFER_QTY:
                break
                
            shortage_store = shortage_entry['Код_магазина']
            shortage_amount = shortage_entry['Недостающий_товар_шт']
            
            if shortage_amount >= MIN_TRANSFER_QTY:
                transfer_amount = min(excess_amount, shortage_amount)
                if transfer_amount >= MIN_TRANSFER_QTY:
                    # Проверяем, сколько уже перемещено из этого магазина для этого товара
                    current_transferred = total_transferred_per_item_store[key]
                    potential_transferred = current_transferred + transfer_amount
                    remaining_stock = initial_stock - potential_transferred
                    
                    # Проверяем остаток после перемещения для товаров со Статус = 1
                    if status == 1 and remaining_stock < MIN_STOCK_ACTIVE:
                        max_transfer = initial_stock - MIN_STOCK_ACTIVE - current_transferred
                        if max_transfer < MIN_TRANSFER_QTY:
                            break
                        transfer_amount = min(transfer_amount, max_transfer)
                        remaining_stock = initial_stock - (current_transferred + transfer_amount)
                    
                    transfer_value = transfer_amount * unit_price
                    recommendations.append({
                        'Товар': item,
                        'Код_номенклатуры': item_code,
                        'От_магазина': excess_store,
                        'В_магазин': shortage_store,
                        'Количество_шт': transfer_amount,
                        'Сумма_перемещения_тыс_руб': transfer_value,
                        'Остаток_после_перемещения_шт': remaining_stock  # Временный остаток, будет пересчитан позже
                    })
                    total_transferred_per_item_store[key] += transfer_amount
                    excess_amount -= transfer_amount
                    shortage_entry['Недостающий_товар_шт'] -= transfer_amount
        
        # Если после перераспределения остался излишек, добавляем его в remaining_excess
        if excess_amount > 0 and excess_store not in target_stores:
            remaining_excess.append({
                'Товар': item,
                'Код_магазина': excess_store,
                'Код_номенклатуры': item_code,
                'Количество_не_перемещено_шт': excess_amount,
                'Сумма_не_перемещено_тыс_руб': excess_amount * unit_price,
                'Статус': status
            })

# 9. Формируем DataFrame для перемещений и неперемещённого товара
recommendations_df = pd.DataFrame(recommendations)
remaining_excess_df = pd.DataFrame(remaining_excess)

# Статистика по перемещённому товару
if not recommendations_df.empty:
    total_moved_qty = recommendations_df['Количество_шт'].sum()
    total_moved_value = recommendations_df['Сумма_перемещения_тыс_руб'].sum()
else:
    total_moved_qty = 0
    total_moved_value = 0

# Статистика по неперемещённому товару
if not remaining_excess_df.empty:
    total_not_moved_qty = remaining_excess_df['Количество_не_перемещено_шт'].sum()
    total_not_moved_value = remaining_excess_df['Сумма_не_перемещено_тыс_руб'].sum()
else:
    total_not_moved_qty = 0
    total_not_moved_value = 0

print("\nСтатистика по перемещённому товару:")
print(f"Количество перемещённого товара (шт.): {total_moved_qty}")
print(f"Сумма перемещённого товара (тыс. руб.): {total_moved_value:.2f}")

print("\nСтатистика по неперемещённому товару:")
print(f"Количество неперемещённого товара (шт.): {total_not_moved_qty}")
print(f"Сумма неперемещённого товара (тыс. руб.): {total_not_moved_value:.2f}")

# 10. Распределяем ВЕСЬ неперемещённый товар на указанные магазины по потребности
# Фильтруем данные только для указанных магазинов
target_df = df[df['Код_магазина'].isin(target_stores)].copy()

# Проверяем, есть ли магазины для распределения
if target_df.empty:
    raise ValueError("Список целевых магазинов пуст. Распределение невозможно.")
else:
    # Распределяем ВЕСЬ неперемещённый товар с учётом потребности
    redistribution = []
    final_remainder = []
    MIN_TRANSFER_QTY = 3
    MAX_PER_STORE = 30
    MIN_STOCK_ACTIVE = 6

    # Словарь для отслеживания общего количества товара одного SKU в каждом магазине
    total_per_store_per_sku = {}

    # Словарь для отслеживания остатка в магазинах-источниках
    remaining_stock_per_store = {}

    for _, excess_row in remaining_excess_df.iterrows():
        item = excess_row['Товар']
        from_store = excess_row['Код_магазина']
        item_code = excess_row['Код_номенклатуры']
        total_to_redistribute = int(excess_row['Количество_не_перемещено_шт'])
        unit_price = excess_row['Сумма_не_перемещено_тыс_руб'] / total_to_redistribute if total_to_redistribute > 0 else 0
        status = excess_row['Статус']
        
        # Инициализируем остаток для текущего магазина-источника
        key = (item_code, from_store)
        if key not in remaining_stock_per_store:
            # Учитываем уже перемещённое количество из шага 8
            already_transferred = total_transferred_per_item_store.get(key, 0)
            remaining_stock_per_store[key] = total_to_redistribute + already_transferred
        
        if total_to_redistribute < MIN_TRANSFER_QTY:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        # Фильтруем target_df по текущему товару
        item_target_df = target_df[target_df['Товар'] == item].copy()
        
        if item_target_df.empty:
            final_remainder.append({
                'Товар': item,
                'Код_номенклатуры': item_code,
                'Из_магазина': from_store,
                'Количество_остаток_шт': total_to_redistribute,
                'Сумма_остаток_тыс_руб': total_to_redistribute * unit_price
            })
            continue
        
        # Сортируем магазины по Запас_в_днях
        store_needs = item_target_df.groupby('Код_магазина').agg({
            'Запас_в_днях': 'mean',
            'Недостающий_товар_шт': 'sum',
            'Ср_дневн_продажи_шт': 'mean'
        }).reset_index()
        store_needs = store_needs.sort_values(by=['Запас_в_днях', 'Недостающий_товар_шт'], ascending=[True, False]).reset_index(drop=True)
        
        remaining_to_distribute = total_to_redistribute
        distributed_amounts = {store: 0 for store in store_needs['Код_магазина']}
        
        store_list = [
            (row['Код_магазина'], row['Запас_в_днях'], row['Недостающий_товар_шт'])
            for _, row in store_needs.iterrows()
        ]
        
        if item_code not in total_per_store_per_sku:
            total_per_store_per_sku[item_code] = {store: 0 for store in target_stores}
        
        # Шаг 1: Распределяем по магазинам с наименьшим запасом
        store_index = 0
        num_stores = len(store_list)
        while remaining_to_distribute >= MIN_TRANSFER_QTY and store_index < num_stores:
            store, stock_days, shortage = store_list[store_index]
            
            current_total_in_store = total_per_store_per_sku[item_code][store]
            if current_total_in_store >= MAX_PER_STORE:
                store_index += 1
                continue
            
            remaining_shortage = max(0, shortage - distributed_amounts[store])
            if remaining_shortage <= 0:
                store_index += 1
                continue
            
            max_allowed_to_store = MAX_PER_STORE - current_total_in_store
            if max_allowed_to_store < MIN_TRANSFER_QTY:
                store_index += 1
                continue
            
            amount = min(remaining_to_distribute, remaining_shortage, max_allowed_to_store)
            if amount < MIN_TRANSFER_QTY:
                store_index += 1
                continue
            
            # Проверяем остаток после распределения для товаров со Статус = 1
            current_transferred = total_transferred_per_item_store.get(key, 0)
            potential_transferred = current_transferred + amount
            remaining_stock = remaining_stock_per_store[key] - potential_transferred
            if status == 1 and remaining_stock < MIN_STOCK_ACTIVE:
                max_distribute = remaining_stock_per_store[key] - MIN_STOCK_ACTIVE - current_transferred
                if max_distribute < MIN_TRANSFER_QTY:
                    store_index += 1
                    continue
                amount = min(amount, max_distribute)
                remaining_stock = remaining_stock_per_store[key] - (current_transferred + amount)
            
            distributed_amounts[store] += amount
            total_per_store_per_sku[item_code][store] += amount
            total_transferred_per_item_store[key] = total_transferred_per_item_store.get(key, 0) + amount
            remaining_to_distribute -= amount
            if remaining_shortage <= amount or total_per_store_per_sku[item_code][store] >= MAX_PER_STORE:
                store_index += 1
        
        # Шаг 2: Распределяем остатки по магазинам с наименьшим запасом
        if remaining_to_distribute >= MIN_TRANSFER_QTY:
            remaining_stores = [
                (store, stock_days)
                for store, stock_days, _ in store_list
                if total_per_store_per_sku[item_code][store] < MAX_PER_STORE
            ]
            remaining_stores.sort(key=lambda x: x[1])
            
            store_index = 0
            num_stores = len(remaining_stores)
            while remaining_to_distribute >= MIN_TRANSFER_QTY and num_stores > 0:
                store, _ = remaining_stores[store_index]
                current_total_in_store = total_per_store_per_sku[item_code][store]
                
                if current_total_in_store >= MAX_PER_STORE:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                max_allowed_to_store = MAX_PER_STORE - current_total_in_store
                if max_allowed_to_store < MIN_TRANSFER_QTY:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                amount = min(remaining_to_distribute, max_allowed_to_store)
                if amount < MIN_TRANSFER_QTY:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                    continue
                
                # Проверяем остаток после распределения для товаров со Статус = 1
                current_transferred = total_transferred_per_item_store.get(key, 0)
                potential_transferred = current_transferred + amount
                remaining_stock = remaining_stock_per_store[key] - potential_transferred
                if status == 1 and remaining_stock < MIN_STOCK_ACTIVE:
                    max_distribute = remaining_stock_per_store[key] - MIN_STOCK_ACTIVE - current_transferred
                    if max_distribute < MIN_TRANSFER_QTY:
                        remaining_stores.pop(store_index)
                        num_stores -= 1
                        if num_stores > 0:
                            store_index = store_index % num_stores
                        continue
                    amount = min(amount, max_distribute)
                    remaining_stock = remaining_stock_per_store[key] - (current_transferred + amount)
                
                distributed_amounts[store] += amount
                total_per_store_per_sku[item_code][store] += amount
                total_transferred_per_item_store[key] = total_transferred_per_item_store.get(key, 0) + amount
                remaining_to_distribute -= amount
                
                if total_per_store_per_sku[item_code][store] >= MAX_PER_STORE:
                    remaining_stores.pop(store_index)
                    num_stores -= 1
                    if num_stores > 0:
                        store_index = store_index % num_stores
                else:
                    store_index = (store_index + 1) % num_stores
        
        # Шаг 3: Формируем записи для распределения
        for to_store, amount in distributed_amounts.items():
            if amount >= MIN_TRANSFER_QTY:
                value = amount * unit_price
                remaining_stock = remaining_stock_per_store[key] - total_transferred_per_item_store.get(key, 0)
                redistribution.append({
                    'Товар': item,
                    'Код_номенклатуры': item_code,
                    'Из_магазина': from_store,
                    'В_магазин': to_store,
                    'Количество_распределено_шт': amount,
                    'Сумма_распределено_тыс_руб': value,
                    'Остаток_после_распределения_шт': remaining_stock
                })

    # Создаём DataFrame для распределённого товара
    redistribution_df = pd.DataFrame(redistribution)

    # Проверяем, что ни один магазин не получил более 18 единиц
    if not redistribution_df.empty:
        store_totals = redistribution_df.groupby(['Код_номенклатуры', 'В_магазин'])['Количество_распределено_шт'].sum().reset_index()
        over_limit = store_totals[store_totals['Количество_распределено_шт'] > MAX_PER_STORE]
        if not over_limit.empty:
            raise ValueError(f"Ошибка: Некоторые магазины получили более {MAX_PER_STORE} единиц товара:\n{over_limit}")

# 11. Объединяем данные из recommendations_df и redistribution_df
# Переименовываем столбцы для единообразия
recommendations_df = recommendations_df.rename(columns={
    'От_магазина': 'От_магазина',
    'В_магазин': 'В_магазин',
    'Количество_шт': 'Кол-во Перемещения',
    'Сумма_перемещения_тыс_руб': 'Сумма_перемещения_тыс_руб',
    'Остаток_после_перемещения_шт': 'Остаток_после_перемещения_шт_где_забрали'
})

redistribution_df = redistribution_df.rename(columns={
    'Из_магазина': 'От_магазина',
    'В_магазин': 'В_магазин',
    'Количество_распределено_шт': 'Кол-во Распределение_неперемещённого',
    'Сумма_распределено_тыс_руб': 'Сумма_распределено_тыс_руб',
    'Остаток_после_распределения_шт': 'Остаток_после_распределения_шт_где_забрали'
})

# Объединяем обе таблицы по Товар, Код_номенклатуры, От_магазина и В_магазин
combined_df = pd.merge(
    recommendations_df,
    redistribution_df,
    on=['Товар', 'Код_номенклатуры', 'От_магазина', 'В_магазин'],
    how='outer'
)

# Заполняем пропуски нулями
combined_df['Кол-во Перемещения'] = combined_df['Кол-во Перемещения'].fillna(0)
combined_df['Кол-во Распределение_неперемещённого'] = combined_df['Кол-во Распределение_неперемещённого'].fillna(0)
combined_df['Сумма_перемещения_тыс_руб'] = combined_df['Сумма_перемещения_тыс_руб'].fillna(0)
combined_df['Сумма_распределено_тыс_руб'] = combined_df['Сумма_распределено_тыс_руб'].fillna(0)

# Суммируем суммы перемещений
combined_df['Сумма_перемещения_тыс_руб'] = combined_df['Сумма_перемещения_тыс_руб'] + combined_df['Сумма_распределено_тыс_руб']

# Пересчитываем остаток в магазине-источнике на основе общего перемещённого количества
combined_df['Остаток_после_перемещения_шт_где_забрали'] = 0
for (item_code, from_store), group in combined_df.groupby(['Код_номенклатуры', 'От_магазина']):
    # Изначальный остаток в магазине-источнике
    initial_stock = df[(df['Код_магазина'] == from_store) & (df['Код_номенклатуры'] == item_code)]['Остатки_шт'].iloc[0]
    # Общее перемещённое количество
    total_transferred = group['Кол-во Перемещения'].sum() + group['Кол-во Распределение_неперемещённого'].sum()
    final_remaining_stock = initial_stock - total_transferred
    # Обновляем остаток для всех строк группы
    combined_df.loc[(combined_df['Код_номенклатуры'] == item_code) & (combined_df['От_магазина'] == from_store), 'Остаток_после_перемещения_шт_где_забрали'] = final_remaining_stock

# Удаляем лишний столбец
combined_df = combined_df.drop(columns=['Сумма_распределено_тыс_руб', 'Остаток_после_распределения_шт_где_забрали'])

# Добавляем остаток в магазине-получателе
combined_df['Остаток_после_перемещения_шт_куда_переместили'] = 0
for idx, row in combined_df.iterrows():
    to_store = row['В_магазин']
    item_code = row['Код_номенклатуры']
    # Изначальный остаток в магазине-получателе
    initial_stock = df[(df['Код_магазина'] == to_store) & (df['Код_номенклатуры'] == item_code)]['Остатки_шт']
    if not initial_stock.empty:
        initial_stock = initial_stock.iloc[0]
    else:
        initial_stock = 0
    # Прибавляем перемещённое количество
    total_received = row['Кол-во Перемещения'] + row['Кол-во Распределение_неперемещённого']
    combined_df.at[idx, 'Остаток_после_перемещения_шт_куда_переместили'] = initial_stock + total_received

# Добавляем статусы товара
combined_df['Статус_товара_в_магазине_где_забрали_товар'] = 0
combined_df['Статус_товара_в_магазине_куда_переместили_товар'] = 0
for idx, row in combined_df.iterrows():
    from_store = row['От_магазина']
    to_store = row['В_магазин']
    item_code = row['Код_номенклатуры']
    
    # Статус в магазине-источнике
    source_status = df[(df['Код_магазина'] == from_store) & (df['Код_номенклатуры'] == item_code)]['Статус']
    if not source_status.empty:
        combined_df.at[idx, 'Статус_товара_в_магазине_где_забрали_товар'] = source_status.iloc[0]
    
    # Статус в магазине-получателе
    receiver_status = df[(df['Код_магазина'] == to_store) & (df['Код_номенклатуры'] == item_code)]['Статус']
    if not receiver_status.empty:
        combined_df.at[idx, 'Статус_товара_в_магазине_куда_переместили_товар'] = receiver_status.iloc[0]

# Формируем итоговый DataFrame с нужными столбцами
final_df = combined_df[[
    'Товар', 'Код_номенклатуры', 'От_магазина', 'В_магазин',
    'Кол-во Перемещения', 'Кол-во Распределение_неперемещённого',
    'Сумма_перемещения_тыс_руб', 'Остаток_после_перемещения_шт_где_забрали',
    'Остаток_после_перемещения_шт_куда_переместили',
    'Статус_товара_в_магазине_где_забрали_товар',
    'Статус_товара_в_магазине_куда_переместили_товар'
]]

# 12. Сохраняем результаты в Excel (только одна вкладка)
with pd.ExcelWriter('recommendations4.xlsx', engine='openpyxl') as writer:
    final_df.to_excel(writer, sheet_name='Перемещения_и_Распределение', index=False)

# Выводим время выполнения
end_time = time.time()
print(f"Время выполнения: {end_time - start_time:.2f} секунд")